# Exploring the Impact of Evaluation Order on Edit Distance Algorithms

Removing data-dependencies in the Wagner-Fisher, Needleman-Wunsch, Smith-Waterman, and Gotoh Dynamic Programming algorithms to explain the hardware-accelerated variants in StringZilla.

## Levenshtein Distance

Levenshtein edit distance is one of the most broadly studied string similarity metrics.
It is defined as the minimum number of single-character insertions, deletions, and substitutions required to change one string into another.
The Levenshtein distance between two strings is calculated using dynamic programming algorithms, such as the Wagner-Fisher algorithm, and its variations for Bioinformatics: 

- Needleman-Wunsch for global alignment with substitution matrices, 
- Smith-Waterman for local alignment with substitution matrices, 
- Gotoh for different penalties for gap opening and extensions.

Given the shared nature of these algorithms, the same tricks can be applied to all of them to improve their performance.

## Warner-Fisher Algorithm

Wagner-Fisher algorithm, in its most naive form, has a time and space complexity of $O(NM)$, where $N$ and $M$ are the lengths of the two strings being compared.
A rectangular matrix of size $(N+1) \times (M+1)$ is created to store the edit distances between all prefixes of the two strings.
The first row and column are, naturally, initialized with ${0, 1, 2, ..., N}$ and ${0, 1, 2, ..., M}$ respectively.

In [1]:
from typing import Tuple
import numpy as np # NumPy for matrices

def wagner_fisher(s1: str, s2: str) -> Tuple[int, np.ndarray]:
    # Create a matrix of size (len(s1)+1) x (len(s2)+1)
    matrix = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)

    # Initialize the first column and first row of the matrix
    for i in range(len(s1) + 1):
        matrix[i, 0] = i
    for j in range(len(s2) + 1):
        matrix[0, j] = j

    # Compute Levenshtein distance
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            substitution_cost = s1[i - 1] != s2[j - 1]
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,                      #? Deletion cost
                matrix[i, j - 1] + 1,                      #? Insertion cost
                matrix[i - 1, j - 1] + substitution_cost,  #? Substitution cost
            )

    # The distance will be placed in the bottom right corner of the matrix
    return matrix[len(s1), len(s2)], matrix

This algorithm is almost never recommended for practical use, as it has a quadratic space complexity.
It's trivial to see that the space complexity can be reduced to $O(min(N, M))$ by only storing the last two rows of the matrix, but we want to keep the entire matrix as a reference to allow debugging and visualization.

## Diagonal Evaluation Order

Accelerating this exact algorithm with SIMD instructions isn't trivial, is the `matrix[i, j]` value has a dependency on the `matrix[i, j - 1]` value.
So we can't brute-force accelerate the inner loop.
Instead, we can show that we can evaluate the matrix in a different order, and still get the same result.

![Skewed Diagonals Evaluation Order](https://mathworld.wolfram.com/images/eps-svg/SkewDiagonal_1000.svg)

But before complicating things too much, let's start with a simple case - when both strings have identical lengths and the DP matrix has a square shape.

In [2]:
def square_skewed_diagonals(s1: str, s2: str, verbose: bool = False) -> Tuple[int, np.ndarray]:
    assert len(s1) == len(s2), "First define an algo for square matrices!"
    # Create a matrix of size (len(s1)+1) x (len(s2)+1)
    matrix = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
    matrix[:, :] = 99

    # Initialize the first column and first row of the matrix
    for i in range(len(s1) + 1):
        matrix[i, 0] = i
    for j in range(len(s2) + 1):
        matrix[0, j] = j

    # Number of rows and columns in the square matrix.
    n = len(s1) + 1
    
    # Number of diagonals and skewed diagonals in the square matrix of size (n x n).
    skew_diagonals_count = 2 * n - 1
    
    # Populate the matrix in 2 separate loops: for the top left triangle and for the bottom right triangle.
    for skew_diagonal_idx in range(2, n):
        skew_diagonal_length = skew_diagonal_idx + 1
        for offset_within_skew_diagonal in range(1, skew_diagonal_length - 1):
            # If we haven't passed the main skew diagonal yet, 
            # then we have to skip the first and the last operation,
            # as those are already pre-populated and form the first column 
            # and the first row of the Levenshtein matrix respectively.
            i = skew_diagonal_idx - offset_within_skew_diagonal
            j = offset_within_skew_diagonal
            if verbose:
                print(f"top left triangle: {skew_diagonal_idx=}, {skew_diagonal_length=}, {i=}, {j=}")
            substitution_cost = s1[i - 1] != s2[j - 1]
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,                      #? Deletion cost
                matrix[i, j - 1] + 1,                      #? Insertion cost
                matrix[i - 1, j - 1] + substitution_cost,  #? Substitution cost
            )
            
    # Now the bottom right triangle of the matrix.
    for skew_diagonal_idx in range(n, skew_diagonals_count):
        skew_diagonal_length = 2*n - skew_diagonal_idx - 1
        for offset_within_skew_diagonal in range(skew_diagonal_length):
            i = n - offset_within_skew_diagonal - 1
            j = skew_diagonal_idx - n + offset_within_skew_diagonal + 1
            if verbose:
                print(f"bottom right triangle: {skew_diagonal_idx=}, {skew_diagonal_length=}, {i=}, {j=}")
            substitution_cost = s1[i - 1] != s2[j - 1]
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,                      #? Deletion cost
                matrix[i, j - 1] + 1,                      #? Insertion cost
                matrix[i - 1, j - 1] + substitution_cost,  #? Substitution cost
            )

    # Similarly, the distance will be placed in the bottom right corner of the matrix
    return matrix[len(s1), len(s2)], matrix

Let's generate some random strings and make sure we produce the right result.

In [3]:
import random
for _ in range(10):
    s1 = ''.join(random.choices("ab", k=50))
    s2 = ''.join(random.choices("ab", k=50))
    d0, _ = wagner_fisher(s1, s2)
    d1, _ = square_skewed_diagonals(s1, s2)
    assert d0 == d1

Going further, we can avoid storing the whole matrix, and only store two diagonals at a time.
The longer will never exceed N. The shorter one is always at most N-1, and is always shorter by one.

In [4]:
s1 = "listen"
s2 = "silent"
# s1 = ''.join(random.choices("abcd", k=100))
# s2 = ''.join(random.choices("abcd", k=100))
distance, baseline = wagner_fisher(s1, s2)
s1, s2, f"{distance = }", baseline

('listen',
 'silent',
 'distance = np.int64(4)',
 array([[0, 1, 2, 3, 4, 5, 6],
        [1, 1, 2, 2, 3, 4, 5],
        [2, 2, 1, 2, 3, 4, 5],
        [3, 2, 2, 2, 3, 4, 5],
        [4, 3, 3, 3, 3, 4, 4],
        [5, 4, 4, 4, 3, 4, 5],
        [6, 5, 5, 5, 4, 3, 4]]))

In [5]:
assert len(s1) == len(s2), "First define an algo for square matrices!"
# Number of rows and columns in the square matrix.
n = len(s1) + 1

# Let's use just a couple of arrays to store the previous skew diagonals.
# Let's imagine that our Levenshtein matrix is gonna have 5x5 size for two words of length 4.
#         B C D E << s2 characters: BCDE
#     + ---------
#     | a b c d e
#   F | f g h i j
#   K | k l m n o
#   P | p q r s t
#   U | u v w x y
#   ^
#   ^ s1 characters: FKPU
following = np.zeros(n, dtype=np.uint) # let's assume we are computing the main skew diagonal: [u, q, m, i, e]
current = np.zeros(n, dtype=np.uint) # will contain: [p, l, h, e]
previous = np.zeros(n, dtype=np.uint) # will contain: [k, g, c]

# Initialize the first two diagonals.
# The `previous` would contain the values [a].
# The `current` would contain the values [f, b]. 
previous[0] = 0
current[0:2] = 1
previous, current, following

(array([0, 0, 0, 0, 0, 0, 0], dtype=uint64),
 array([1, 1, 0, 0, 0, 0, 0], dtype=uint64),
 array([0, 0, 0, 0, 0, 0, 0], dtype=uint64))

To feel safer, while designing our alternative traversal algorithm, let's define an extraction function, that will get the values of a certain skewed diagonal.

In [6]:
def get_skewed_diagonal(matrix: np.ndarray, index: int):
    flipped_matrix = np.fliplr(matrix)
    return np.flip(np.diag(flipped_matrix, k= matrix.shape[1] - index - 1))

In [7]:
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])
assert np.all(get_skewed_diagonal(matrix, 2) == [7, 5, 3])
assert np.all(get_skewed_diagonal(matrix, 1) == [4, 2])
assert np.all(get_skewed_diagonal(matrix, 4) == [9])

In [8]:
# To evaluate every subsequent entry:
following_skew_diagonal_idx = 2
while following_skew_diagonal_idx < n:
    following_skew_diagonal_length = following_skew_diagonal_idx + 1

    old_substitution_costs = previous[:following_skew_diagonal_length - 2]
    added_substitution_costs = [s1[following_skew_diagonal_idx - i - 2] != s2[i] for i in range(following_skew_diagonal_length - 2)]
    substitution_costs = old_substitution_costs + added_substitution_costs

    following[1:following_skew_diagonal_length-1] = np.minimum(current[1:following_skew_diagonal_length-1] + 1, current[:following_skew_diagonal_length-2] + 1) # Insertions or deletions
    following[1:following_skew_diagonal_length-1] = np.minimum(following[1:following_skew_diagonal_length-1], substitution_costs) # Substitutions
    following[0] = following_skew_diagonal_idx
    following[following_skew_diagonal_length-1] = following_skew_diagonal_idx
    assert np.all(following[:following_skew_diagonal_length] == get_skewed_diagonal(baseline, following_skew_diagonal_idx))
    
    previous[:] = current[:]
    current[:] = following[:]
    following_skew_diagonal_idx += 1

previous, current, following # Log the state

(array([5, 3, 2, 2, 3, 5, 0], dtype=uint64),
 array([6, 4, 3, 2, 3, 4, 6], dtype=uint64),
 array([6, 4, 3, 2, 3, 4, 6], dtype=uint64))

By now we've scanned through the upper triangle of the matrix, where each subsequent iteration results in a larger diagonal. From now onwards, we will be shrinking. Instead of adding value equal to the skewed diagonal index on either side, we will be cropping those values out.

In [9]:
while following_skew_diagonal_idx < 2 * n - 1:
    following_skew_diagonal_length = 2 * n - 1 - following_skew_diagonal_idx
    old_substitution_costs = previous[:following_skew_diagonal_length]
    added_substitution_costs = [s1[len(s1) - i - 1] != s2[following_skew_diagonal_idx - n + i] for i in range(following_skew_diagonal_length)]
    substitution_costs = old_substitution_costs + added_substitution_costs
    
    following[:following_skew_diagonal_length] = np.minimum(current[:following_skew_diagonal_length] + 1, current[1:following_skew_diagonal_length+1] + 1) # Insertions or deletions
    following[:following_skew_diagonal_length] = np.minimum(following[:following_skew_diagonal_length], substitution_costs) # Substitutions
    assert np.all(following[:following_skew_diagonal_length] == get_skewed_diagonal(baseline, following_skew_diagonal_idx)), f"\n{following[:following_skew_diagonal_length]} not equal to \n{get_skewed_diagonal(baseline, following_skew_diagonal_idx)}"
    
    previous[:following_skew_diagonal_length] = current[1:following_skew_diagonal_length+1]
    current[:following_skew_diagonal_length] = following[:following_skew_diagonal_length]
    following_skew_diagonal_idx += 1

previous, current, following # Log the state

(array([5, 4, 5, 5, 5, 6, 0], dtype=uint64),
 array([4, 5, 4, 5, 5, 5, 6], dtype=uint64),
 array([4, 5, 4, 5, 5, 5, 6], dtype=uint64))

In [10]:
assert distance == following[0], f"{distance = } != {following[0] = }"

## Generalizing to Non-Square Matrices

In [11]:
def skewed_diagonals(s1, s2, verbose: bool = False) -> int:
    shorter, longer = (s1, s2) if len(s1) < len(s2) else (s2, s1)    
    shorter_dim = len(shorter) + 1
    longer_dim = len(longer) + 1
    # Create a matrix of size (len(s1)+1) x (len(s2)+1)
    matrix = np.zeros((len(shorter) + 1, len(longer) + 1), dtype=int)
    matrix[:, :] = 99

    # Initialize the first column and first row of the matrix
    for i in range(shorter_dim):
        matrix[i, 0] = i
    for j in range(longer_dim):
        matrix[0, j] = j

    # Let's say we are dealing with 6 and 9 letter words.
    # The matrix will have size 7 x 10, parameterized as (shorter_dim x longer_dim).
    # It will have:
    # - 8 diagonals of increasing length, at positions: 0, 1, 2, 3, 4, 5, 6, 7.
    # - 2 diagonals of fixed length, at positions: 8, 9.
    # - 8 diagonals of decreasing length, at positions: 10, 11, 12, 13, 14, 15, 16, 17.
    skew_diagonals_count = 2 * longer_dim - 1

    # Same as with square matrices, the 0th diagonal contains - just one element - zero - skipping it.
    # Same as with square matrices, the 1st diagonal contains the values 1 and 1 - skipping it.
    # Now let's handle the rest of the upper triangle.
    for skew_diagonal_idx in range(2, shorter_dim + 1):
        skew_diagonal_length = (skew_diagonal_idx + 1)
        for offset_within_skew_diagonal in range(1, skew_diagonal_length-1): #! Skip the first column & row
            # If we haven't passed the main skew diagonal yet, 
            # then we have to skip the first and the last operation,
            # as those are already pre-populated and form the first column 
            # and the first row of the Levenshtein matrix respectively.
            i = skew_diagonal_idx - offset_within_skew_diagonal
            j = offset_within_skew_diagonal
            if verbose:
                print(f"top left triangle: {skew_diagonal_idx=}, {skew_diagonal_length=}, {i=}, {j=}")
            shorter_char = shorter[i - 1]
            longer_char = longer[j - 1]
            substitution_cost = shorter_char != longer_char
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,  # Deletion
                matrix[i, j - 1] + 1,  # Insertion
                matrix[i - 1, j - 1] + substitution_cost,  # Substitution
            )
            
    # Now let's handle the anti-diagonal band of the matrix, between the top and bottom triangles.        
    for skew_diagonal_idx in range(shorter_dim + 1, longer_dim + 1):
        skew_diagonal_length = shorter_dim
        for offset_within_skew_diagonal in range(skew_diagonal_length):
            i = shorter_dim - offset_within_skew_diagonal - 1
            j = offset_within_skew_diagonal + 1
            if verbose:
                print(f"anti-band: {skew_diagonal_idx=}, {skew_diagonal_length=}, {i=}, {j=}")
            shorter_char = shorter[i - 1]
            longer_char = longer[j - 1]
            substitution_cost = shorter_char != longer_char
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,  # Deletion
                matrix[i, j - 1] + 1,  # Insertion
                matrix[i - 1, j - 1] + substitution_cost,  # Substitution
            )
    
    # Now let's handle the bottom right triangle.
    for skew_diagonal_idx in range(longer_dim + 1, skew_diagonals_count):
        skew_diagonal_length = 2 * longer_dim - skew_diagonal_idx - 1
        for offset_within_skew_diagonal in range(skew_diagonal_length):
            i = shorter_dim - offset_within_skew_diagonal - 1
            j = skew_diagonal_idx - longer_dim + offset_within_skew_diagonal + 1
            if verbose:
                print(f"bottom right triangle: {skew_diagonal_idx=}, {skew_diagonal_length=}, {i=}, {j=}")
            shorter_char = shorter[i - 1]
            longer_char = longer[j - 1]
            substitution_cost = shorter_char != longer_char
            matrix[i, j] = min(
                matrix[i - 1, j] + 1,  # Deletion
                matrix[i, j - 1] + 1,  # Insertion
                matrix[i - 1, j - 1] + substitution_cost,  # Substitution
            )

    # Return the Levenshtein distance
    return matrix[len(shorter), len(longer)], matrix

In [12]:
s1 = "listeners"
s2 = "silents"
distance, baseline = skewed_diagonals(s1, s2)
s1, s2, f"{distance = }", baseline

('listeners',
 'silents',
 'distance = np.int64(5)',
 array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [1, 1, 2, 2, 3, 4, 5, 6, 7, 8],
        [2, 2, 1, 2, 3, 4, 5, 6, 7, 8],
        [3, 2, 2, 2, 3, 4, 5, 6, 7, 8],
        [4, 3, 3, 3, 3, 3, 4, 5, 6, 7],
        [5, 4, 4, 4, 4, 4, 3, 4, 5, 6],
        [6, 5, 5, 5, 4, 5, 4, 4, 5, 6],
        [7, 6, 6, 5, 5, 5, 5, 5, 5, 5]]))

In [13]:
distance, baseline = wagner_fisher(s1, s2)
s1, s2, f"{distance = }", baseline

('listeners',
 'silents',
 'distance = np.int64(5)',
 array([[0, 1, 2, 3, 4, 5, 6, 7],
        [1, 1, 2, 2, 3, 4, 5, 6],
        [2, 2, 1, 2, 3, 4, 5, 6],
        [3, 2, 2, 2, 3, 4, 5, 5],
        [4, 3, 3, 3, 3, 4, 4, 5],
        [5, 4, 4, 4, 3, 4, 5, 5],
        [6, 5, 5, 5, 4, 3, 4, 5],
        [7, 6, 6, 6, 5, 4, 4, 5],
        [8, 7, 7, 7, 6, 5, 5, 5],
        [9, 8, 8, 8, 7, 6, 6, 5]]))

In [14]:
s1 = ''.join(random.choices("abcd", k=5))
s2 = ''.join(random.choices("abcd", k=6))
distance_v0, baseline_v0 = wagner_fisher(s1, s2)
distance_v2, baseline_v2 = skewed_diagonals(s1, s2, verbose=False)
assert distance_v0 == distance_v2, f"{distance_v0 = } != {distance_v2 = }"
assert np.all(baseline_v0 == baseline_v2), f"{baseline_v0 = }\n{baseline_v2 = }"